In [1]:
import pandas as pd
import numpy as np
import json
from scipy import stats

C:\Users\esmat\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
ft_tasks = ["cola", "sst2", "mrpc", "stsb", "qnli", "rte"]
probing_tasks = ["Length", "Depth", "TopConstituents", "BigramShift", "Tense", "SubjNumber", "ObjNumber", "OddManOut", "CoordinationInversion"]
probing_tasks_short = ["Length", "Depth", "TopConst", "BigramShift", "Tense", "SubjNumber", "ObjNumber", "OddManOut", "CoordInv"]
random_seeds = [1, 10, 42, 123, 1234]
encoders = ["bert", "roberta"]
num_exp = 34

In [3]:
exp_dir = "../experiments"
probing_dict = {}
for enc in encoders:
    probing_dict[enc] = {}
    for rs in random_seeds:
        probing_data = np.zeros((num_exp, len(probing_tasks)))
        for exp in range(0,num_exp):
            file_path = f"{exp_dir}/{enc}/{rs}/{exp}/probe_results.json"
            f = open(file_path)
            probing_res = json.load(f)
            for i, task in enumerate(probing_tasks):
                probing_data[exp,i] = probing_res[task]['acc']
            f.close()
        probing_dict[enc][rs] = probing_data
probing_dict

{'bert': {1: array([[59.3, 27.3, 66.1, 83.2, 88.6, 79.6, 75. , 59.4, 64.8],
         [56. , 27.5, 64. , 89.9, 87.5, 80.6, 76.5, 65.1, 64.3],
         [56.7, 26.5, 62.8, 81.3, 87.7, 78.1, 74.4, 61. , 65.9],
         [56.3, 28.2, 57.2, 86. , 87.1, 78.9, 75.6, 62.9, 64.8],
         [58.9, 30. , 62.7, 82.7, 85.6, 79.4, 77.7, 61.6, 62.8],
         [56.4, 29.5, 68.2, 82.2, 87.4, 84.3, 77.3, 61.1, 67. ],
         [56.7, 29.3, 66.2, 83.1, 88. , 82.9, 77.1, 60.7, 64.9],
         [53.6, 25.5, 63.4, 88.9, 87.8, 81.2, 75. , 64. , 68.3],
         [56.5, 27.4, 63.4, 89.6, 87.2, 79.2, 76.2, 63.9, 65.8],
         [57.7, 24.1, 63.9, 89.7, 88.4, 81.2, 76.7, 62.1, 64.4],
         [59.5, 29.2, 66.7, 89.1, 88.1, 81. , 77. , 63.8, 66.7],
         [65.2, 27.9, 64.2, 88.4, 88.2, 82. , 77.4, 63.8, 66.3],
         [54.4, 29.8, 61. , 81.8, 87.1, 78.8, 76.3, 61. , 66.7],
         [52.1, 26.6, 62.2, 81.6, 86.4, 75. , 73.9, 58.7, 67.6],
         [51.7, 28.1, 61.3, 79.6, 87.2, 79.3, 73.2, 57.9, 64.6],
         [57.7

In [4]:
ind_effect = {
    "cola": [[0,1], [2,7], [3,8], [4,9], [5,10], [6,11], [17,22], [18,23], [19,24], [20,25]],
    "sst2": [[0,2], [1,7], [3,12], [4,13], [5,14], [6,15], [17,26], [18,27], [19,28], [20,29]],
    "mrpc": [[0,3], [4,16], [1,8], [2,12], [5,17], [6,18], [10,22], [11,23], [14,26], [15,27]],
    "stsb": [[0,4], [3,16], [1,9], [2,13], [5,19], [6,20], [10,24], [11,25], [14,28], [15,29]],
    "qnli": [[0,5], [6,21], [1,10], [2,14], [3,17], [4,19], [8,22], [9,24], [12,26], [13,28]],
    "rte": [[0,6], [5,21], [1,11], [2,15], [3,18], [4,20], [8,23], [9,25], [12,27], [13,29]]
}

exps_dict = {
    1: "COLA",
    2: "SST2",
    3: "MRPC",
    4: "STSB",
    5: "QNLI",
    6: "RTE",
    7: "COLA SST2",
    8: "COLA MRPC",
    9: "COLA STSB",
    10: "COLA QNLI",
    11: "COLA RTE",
    12: "SST2 MRPC",
    13: "SST2 STSB",
    14: "SST2 QNLI",
    15: "SST2 RTE",
    16: "MRPC STSB",
    17: "MRPC QNLI",
    18: "MRPC RTE",
    19: "STSB QNLI",
    20: "STSB RTE",
    21: "QNLI RTE",
    22: "COLA MRPC QNLI",
    23: "COLA MRPC RTE",
    24: "COLA STSB QNLI",
    25: "COLA STSB RTE",
    26: "SST2 MRPC QNLI",
    27: "SST2 MRPC RTE",
    28: "SST2 STSB QNLI",
    29: "SST2 STSB RTE",
    30: "COLA SST2 MRPC STSB",
    31: "COLA SST2 QNLI RTE",
    32: "MRPC STSB QNLI RTE",
    33: "COLA SST2 MRPC STSB QNLI RTE"
}

In [5]:
def p_to_stars(p, bonferroni=1):
    if p < 0.001 / bonferroni:
        return "***"
    elif p < 0.01 / bonferroni:
        return "**"
    elif p < 0.05 / bonferroni:
        return "*"
    else:
        return ""

In [6]:
def get_avg_probe_res_diff(probe_res_start, probe_res_finish):
    avg_probe_res_diff = list(np.around(np.mean(np.array(probe_res_finish)-np.array(probe_res_start), axis=0), 2))
    _, p = stats.ttest_ind(probe_res_start, probe_res_finish)
    p = list(np.around(np.array(p), 2))         
    p_star = [p_to_stars(p_val) for p_val in p]            
    for i in range(len(p_star)):
        optional_hspace = ["", "\hspace{0.75em}"][int(avg_probe_res_diff[i]>0)]
        avg_probe_res_diff[i] = f"${optional_hspace}{avg_probe_res_diff[i]}" + "^{" + p_star[i] + "}$"
    return avg_probe_res_diff

In [7]:
ind_effect_task_dfs_dict = {}
for task in ft_tasks:
    task_df_data = []
    for enc in encoders:
        for start, finish in ind_effect[task]:
            probe_res_start = []
            probe_res_finish = []
            for rs in random_seeds:
                probe_res_start += [probing_dict[enc][rs][start]]
                probe_res_finish += [probing_dict[enc][rs][finish]]
            avg_probe_res_diff = get_avg_probe_res_diff(probe_res_start, probe_res_finish)
            task_df_data += [[exps_dict[finish], enc] + avg_probe_res_diff]  
    task_df = pd.DataFrame(data=task_df_data, columns=["exp", "LM"]+probing_tasks_short)
    task_df = task_df.set_index("exp")
    task_df.index.name = None
    ind_effect_task_dfs_dict[task] = task_df

In [8]:
ind_effect_enc_dfs_dict = {}
for enc in encoders:
    enc_df_data = []
    for task in ft_tasks:
        probe_res_start = []
        probe_res_finish = []
        for start, finish in ind_effect[task]:
            for rs in random_seeds:
                probe_res_start += [probing_dict[enc][rs][start]]
                probe_res_finish += [probing_dict[enc][rs][finish]]
        avg_probe_res_diff = get_avg_probe_res_diff(probe_res_start, probe_res_finish)
        enc_df_data += [[task.upper(), enc] + avg_probe_res_diff]  
    enc_df = pd.DataFrame(data=enc_df_data, columns=["task", "LM"]+probing_tasks_short)
    enc_df = enc_df.set_index("task")
    enc_df.index.name = None
    ind_effect_enc_dfs_dict[enc] = enc_df

# Individual Effects

## BERT

In [9]:
bert_df = ind_effect_enc_dfs_dict["bert"]
bert_df

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
COLA,bert,$-0.36^{}$,$-0.47^{}$,$-0.83^{}$,$\hspace{0.75em}6.35^{***}$,$-0.23^{}$,$-0.38^{}$,$-0.69^{*}$,$\hspace{0.75em}1.93^{***}$,$\hspace{0.75em}1.09^{***}$
SST2,bert,$-4.08^{***}$,$-0.51^{}$,$-3.92^{***}$,$-1.99^{***}$,$-0.24^{}$,$-1.75^{***}$,$-1.73^{***}$,$-0.79^{*}$,$\hspace{0.75em}0.06^{}$
MRPC,bert,$\hspace{0.75em}0.65^{}$,$\hspace{0.75em}0.75^{***}$,$-2.34^{***}$,$\hspace{0.75em}0.55^{}$,$-0.45^{***}$,$-0.78^{*}$,$\hspace{0.75em}1.07^{***}$,$\hspace{0.75em}1.13^{***}$,$\hspace{0.75em}1.34^{***}$
STSB,bert,$-0.53^{}$,$\hspace{0.75em}0.27^{}$,$-1.66^{*}$,$-0.24^{}$,$-0.3^{*}$,$\hspace{0.75em}0.36^{}$,$\hspace{0.75em}0.2^{}$,$\hspace{0.75em}0.36^{}$,$-0.09^{}$
QNLI,bert,$\hspace{0.75em}0.05^{}$,$\hspace{0.75em}0.33^{}$,$\hspace{0.75em}0.85^{}$,$-0.49^{}$,$-0.16^{}$,$\hspace{0.75em}0.81^{*}$,$\hspace{0.75em}0.16^{}$,$-0.34^{}$,$\hspace{0.75em}1.33^{***}$
RTE,bert,$\hspace{0.75em}0.83^{}$,$\hspace{0.75em}0.34^{}$,$-0.58^{}$,$-0.24^{}$,$-0.21^{}$,$-0.15^{}$,$-0.65^{*}$,$\hspace{0.75em}0.16^{}$,$\hspace{0.75em}0.64^{}$


## RoBERTa

In [10]:
roberta_df = ind_effect_enc_dfs_dict["roberta"]
roberta_df

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
COLA,roberta,$-2.75^{***}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}2.17^{*}$,$\hspace{0.75em}9.17^{***}$,$\hspace{0.75em}1.21^{***}$,$\hspace{0.75em}0.26^{}$,$\hspace{0.75em}0.59^{}$,$\hspace{0.75em}3.66^{***}$,$\hspace{0.75em}2.0^{***}$
SST2,roberta,$-7.12^{***}$,$-1.86^{***}$,$-5.88^{***}$,$-4.43^{***}$,$-1.57^{***}$,$-3.92^{***}$,$-3.96^{***}$,$-1.85^{***}$,$-2.08^{***}$
MRPC,roberta,$-0.06^{}$,$\hspace{0.75em}0.25^{}$,$-0.87^{}$,$-1.68^{}$,$-1.01^{*}$,$-0.61^{}$,$-0.45^{}$,$\hspace{0.75em}0.07^{}$,$\hspace{0.75em}1.0^{}$
STSB,roberta,$\hspace{0.75em}0.03^{}$,$\hspace{0.75em}0.14^{}$,$-3.17^{***}$,$-3.0^{*}$,$-1.84^{***}$,$-3.11^{***}$,$-1.65^{*}$,$-0.43^{}$,$\hspace{0.75em}0.16^{}$
QNLI,roberta,$-0.67^{}$,$\hspace{0.75em}0.04^{}$,$-0.13^{}$,$-0.57^{}$,$-0.56^{}$,$-0.33^{}$,$-1.47^{*}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}0.13^{}$
RTE,roberta,$\hspace{0.75em}0.78^{}$,$-0.84^{*}$,$-0.58^{}$,$-0.32^{}$,$-0.81^{}$,$-0.85^{}$,$-2.04^{***}$,$-0.25^{}$,$0.0^{}$


# COLA

## BERT

In [11]:
cola_df = ind_effect_task_dfs_dict["cola"]
cola_df_bert = cola_df[cola_df["LM"]=="bert"]
cola_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
COLA,bert,$-1.32^{}$,$\hspace{0.75em}0.8^{}$,$-2.94^{*}$,$\hspace{0.75em}6.32^{***}$,$-1.14^{***}$,$-0.02^{}$,$-0.34^{}$,$\hspace{0.75em}3.56^{***}$,$\hspace{0.75em}0.3^{}$
COLA SST2,bert,$-1.16^{}$,$-1.2^{}$,$-0.0^{}$,$\hspace{0.75em}7.4^{***}$,$-0.7^{}$,$\hspace{0.75em}0.22^{}$,$-0.7^{}$,$\hspace{0.75em}2.22^{***}$,$\hspace{0.75em}1.82^{}$
COLA MRPC,bert,$-0.6^{}$,$\hspace{0.75em}0.78^{}$,$-0.06^{}$,$\hspace{0.75em}5.6^{***}$,$\hspace{0.75em}0.44^{}$,$\hspace{0.75em}0.14^{}$,$-0.18^{}$,$\hspace{0.75em}0.56^{}$,$\hspace{0.75em}0.92^{}$
COLA STSB,bert,$-2.84^{}$,$-1.0^{}$,$-0.8^{}$,$\hspace{0.75em}6.48^{***}$,$\hspace{0.75em}0.94^{}$,$\hspace{0.75em}0.08^{}$,$-1.94^{*}$,$\hspace{0.75em}3.34^{*}$,$\hspace{0.75em}3.54^{***}$
COLA QNLI,bert,$\hspace{0.75em}1.24^{}$,$-1.26^{}$,$-1.26^{}$,$\hspace{0.75em}7.2^{***}$,$-0.24^{}$,$-0.96^{}$,$-0.66^{}$,$\hspace{0.75em}2.46^{}$,$\hspace{0.75em}0.58^{}$
COLA RTE,bert,$\hspace{0.75em}1.5^{}$,$-1.82^{*}$,$-3.3^{*}$,$\hspace{0.75em}5.94^{***}$,$-0.26^{}$,$-2.02^{}$,$-0.68^{}$,$\hspace{0.75em}2.16^{}$,$\hspace{0.75em}0.96^{}$
COLA MRPC QNLI,bert,$\hspace{0.75em}0.62^{}$,$\hspace{0.75em}0.46^{}$,$-2.86^{}$,$\hspace{0.75em}5.22^{***}$,$-1.08^{*}$,$-0.62^{}$,$-0.46^{}$,$\hspace{0.75em}1.12^{}$,$\hspace{0.75em}0.28^{}$
COLA MRPC RTE,bert,$-0.58^{}$,$-0.64^{}$,$\hspace{0.75em}0.92^{}$,$\hspace{0.75em}5.52^{***}$,$-0.32^{}$,$\hspace{0.75em}1.34^{}$,$-0.24^{}$,$\hspace{0.75em}1.46^{***}$,$\hspace{0.75em}1.02^{}$
COLA STSB QNLI,bert,$-0.32^{}$,$-0.52^{}$,$\hspace{0.75em}1.12^{}$,$\hspace{0.75em}6.56^{***}$,$\hspace{0.75em}0.24^{}$,$-1.26^{}$,$-0.44^{}$,$\hspace{0.75em}1.38^{*}$,$\hspace{0.75em}0.38^{}$
COLA STSB RTE,bert,$-0.18^{}$,$-0.26^{}$,$\hspace{0.75em}0.92^{}$,$\hspace{0.75em}7.24^{***}$,$-0.2^{}$,$-0.7^{}$,$-1.22^{}$,$\hspace{0.75em}1.08^{}$,$\hspace{0.75em}1.08^{}$


## RoBERTa

In [12]:
cola_df_roberta = cola_df[cola_df["LM"]=="roberta"]
cola_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
COLA,roberta,$\hspace{0.75em}2.18^{***}$,$\hspace{0.75em}1.24^{}$,$\hspace{0.75em}8.62^{***}$,$\hspace{0.75em}5.78^{***}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}0.44^{}$,$-1.78^{*}$,$\hspace{0.75em}4.64^{***}$,$\hspace{0.75em}2.7^{*}$
COLA SST2,roberta,$-0.66^{}$,$\hspace{0.75em}2.18^{}$,$\hspace{0.75em}6.18^{***}$,$\hspace{0.75em}13.08^{***}$,$\hspace{0.75em}2.44^{}$,$\hspace{0.75em}3.28^{}$,$\hspace{0.75em}2.42^{}$,$\hspace{0.75em}4.4^{***}$,$\hspace{0.75em}4.52^{*}$
COLA MRPC,roberta,$-5.48^{*}$,$-0.34^{}$,$-1.58^{}$,$\hspace{0.75em}7.94^{***}$,$\hspace{0.75em}0.14^{}$,$\hspace{0.75em}0.16^{}$,$\hspace{0.75em}0.78^{}$,$\hspace{0.75em}2.62^{*}$,$\hspace{0.75em}1.6^{}$
COLA STSB,roberta,$-3.84^{*}$,$-1.0^{}$,$\hspace{0.75em}3.8^{}$,$\hspace{0.75em}11.9^{***}$,$\hspace{0.75em}3.46^{*}$,$\hspace{0.75em}0.98^{}$,$-0.1^{}$,$\hspace{0.75em}4.6^{*}$,$\hspace{0.75em}2.2^{}$
COLA QNLI,roberta,$-3.68^{}$,$-0.5^{}$,$\hspace{0.75em}0.38^{}$,$\hspace{0.75em}5.86^{***}$,$-0.1^{}$,$-1.88^{}$,$-0.96^{}$,$\hspace{0.75em}3.84^{***}$,$\hspace{0.75em}3.02^{*}$
COLA RTE,roberta,$-1.82^{}$,$\hspace{0.75em}1.04^{}$,$\hspace{0.75em}6.04^{*}$,$\hspace{0.75em}7.78^{***}$,$\hspace{0.75em}0.36^{}$,$-0.5^{}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}4.18^{***}$,$\hspace{0.75em}4.28^{***}$
COLA MRPC QNLI,roberta,$-5.58^{*}$,$-0.7^{}$,$\hspace{0.75em}0.24^{}$,$\hspace{0.75em}8.8^{***}$,$\hspace{0.75em}0.78^{}$,$\hspace{0.75em}1.0^{}$,$\hspace{0.75em}0.86^{}$,$\hspace{0.75em}2.18^{*}$,$\hspace{0.75em}0.54^{}$
COLA MRPC RTE,roberta,$\hspace{0.75em}0.56^{}$,$\hspace{0.75em}1.34^{*}$,$\hspace{0.75em}3.1^{}$,$\hspace{0.75em}8.64^{***}$,$\hspace{0.75em}3.32^{*}$,$\hspace{0.75em}1.54^{}$,$\hspace{0.75em}2.54^{}$,$\hspace{0.75em}3.3^{*}$,$\hspace{0.75em}0.62^{}$
COLA STSB QNLI,roberta,$-4.04^{}$,$\hspace{0.75em}0.38^{}$,$\hspace{0.75em}0.88^{}$,$\hspace{0.75em}11.02^{***}$,$\hspace{0.75em}1.22^{}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}1.28^{}$,$\hspace{0.75em}3.4^{***}$,$\hspace{0.75em}0.1^{}$
COLA STSB RTE,roberta,$-5.12^{***}$,$-0.8^{}$,$-5.98^{}$,$\hspace{0.75em}10.86^{***}$,$\hspace{0.75em}0.22^{}$,$-2.74^{}$,$\hspace{0.75em}0.62^{}$,$\hspace{0.75em}3.46^{}$,$\hspace{0.75em}0.42^{}$


# SST2

## BERT

In [13]:
sst2_df = ind_effect_task_dfs_dict["sst2"]
sst2_df_bert = sst2_df[sst2_df["LM"]=="bert"]
sst2_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
SST2,bert,$-4.58^{***}$,$\hspace{0.75em}0.3^{}$,$-4.94^{*}$,$-1.86^{***}$,$-0.88^{*}$,$-0.86^{*}$,$0.0^{}$,$\hspace{0.75em}1.22^{***}$,$0.0^{}$
COLA SST2,bert,$-4.42^{*}$,$-1.7^{}$,$-2.0^{}$,$-0.78^{*}$,$-0.44^{}$,$-0.62^{}$,$-0.36^{}$,$-0.12^{}$,$\hspace{0.75em}1.52^{}$
SST2 MRPC,bert,$-3.92^{*}$,$-0.02^{}$,$-2.2^{}$,$-2.42^{*}$,$\hspace{0.75em}0.2^{}$,$-0.26^{}$,$-1.62^{}$,$-0.88^{}$,$\hspace{0.75em}0.16^{}$
SST2 STSB,bert,$-5.28^{*}$,$-0.6^{}$,$-3.44^{}$,$-2.68^{***}$,$\hspace{0.75em}0.24^{}$,$-2.42^{*}$,$-3.66^{***}$,$\hspace{0.75em}0.7^{}$,$\hspace{0.75em}2.8^{*}$
SST2 QNLI,bert,$-4.22^{*}$,$-0.28^{}$,$-4.88^{***}$,$-1.4^{}$,$-0.2^{}$,$-3.16^{*}$,$-2.8^{*}$,$-1.82^{}$,$-1.28^{}$
SST2 RTE,bert,$-2.76^{*}$,$-1.96^{}$,$-6.96^{***}$,$-2.48^{***}$,$-0.24^{}$,$-3.82^{*}$,$-2.0^{*}$,$-1.38^{}$,$-1.3^{}$
SST2 MRPC QNLI,bert,$-2.52^{}$,$\hspace{0.75em}0.34^{}$,$-4.92^{*}$,$-2.06^{*}$,$-0.66^{}$,$-1.9^{}$,$-1.46^{}$,$-1.5^{}$,$-0.58^{}$
SST2 MRPC RTE,bert,$-4.52^{*}$,$-0.84^{}$,$-2.5^{}$,$-2.18^{*}$,$-0.58^{}$,$\hspace{0.75em}0.5^{}$,$-1.58^{}$,$-1.38^{*}$,$-0.36^{}$
SST2 STSB QNLI,bert,$-3.06^{*}$,$\hspace{0.75em}0.74^{}$,$-2.66^{}$,$-1.78^{*}$,$\hspace{0.75em}0.36^{}$,$-2.14^{}$,$-1.36^{}$,$-0.4^{}$,$\hspace{0.75em}0.8^{}$
SST2 STSB RTE,bert,$-5.48^{*}$,$-1.04^{}$,$-4.66^{*}$,$-2.24^{*}$,$-0.22^{}$,$-2.86^{*}$,$-2.48^{*}$,$-2.36^{}$,$-1.12^{}$


## RoBERTa

In [14]:
sst2_df_roberta = sst2_df[sst2_df["LM"]=="roberta"]
sst2_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
SST2,roberta,$-5.44^{***}$,$-2.84^{*}$,$-5.7^{***}$,$-8.76^{***}$,$-4.0^{***}$,$-7.28^{***}$,$-8.06^{***}$,$-2.32^{*}$,$-3.76^{***}$
COLA SST2,roberta,$-8.28^{***}$,$-1.9^{*}$,$-8.14^{***}$,$-1.46^{*}$,$-1.84^{*}$,$-4.44^{*}$,$-3.86^{*}$,$-2.56^{***}$,$-1.94^{}$
SST2 MRPC,roberta,$-9.18^{***}$,$-1.88^{}$,$-8.42^{***}$,$-3.12^{}$,$-2.02^{}$,$-2.88^{}$,$-4.32^{***}$,$-1.44^{}$,$-4.26^{***}$
SST2 STSB,roberta,$-8.3^{***}$,$-2.78^{*}$,$-5.34^{*}$,$-2.12^{}$,$\hspace{0.75em}0.4^{}$,$-3.08^{}$,$-4.46^{***}$,$-0.88^{}$,$-2.32^{*}$
SST2 QNLI,roberta,$-5.26^{*}$,$-1.82^{}$,$-5.52^{}$,$-6.64^{***}$,$-1.36^{}$,$-4.2^{}$,$-4.22^{*}$,$-1.72^{}$,$-0.94^{}$
SST2 RTE,roberta,$-5.64^{***}$,$-1.36^{}$,$\hspace{0.75em}2.44^{}$,$-2.98^{***}$,$\hspace{0.75em}0.68^{}$,$-1.72^{}$,$-2.58^{}$,$-1.82^{}$,$\hspace{0.75em}2.2^{*}$
SST2 MRPC QNLI,roberta,$-7.1^{*}$,$-2.26^{*}$,$-7.94^{***}$,$-4.6^{*}$,$-2.42^{*}$,$-1.8^{}$,$-2.38^{}$,$-3.2^{*}$,$-2.28^{}$
SST2 MRPC RTE,roberta,$-4.44^{*}$,$-0.58^{}$,$-6.96^{*}$,$-4.6^{*}$,$-1.22^{}$,$-6.96^{*}$,$-4.46^{*}$,$-1.46^{}$,$-3.9^{}$
SST2 STSB QNLI,roberta,$-7.84^{*}$,$-0.86^{}$,$-3.5^{}$,$-5.92^{*}$,$-1.7^{}$,$-2.36^{}$,$-2.42^{}$,$-1.96^{}$,$-1.94^{}$
SST2 STSB RTE,roberta,$-9.76^{***}$,$-2.34^{***}$,$-9.72^{*}$,$-4.14^{*}$,$-2.18^{}$,$-4.5^{}$,$-2.88^{}$,$-1.18^{}$,$-1.64^{}$


# MRPC

## BERT

In [15]:
mrpc_df = ind_effect_task_dfs_dict["mrpc"]
mrpc_df_bert = mrpc_df[mrpc_df["LM"]=="bert"]
mrpc_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
MRPC,bert,$\hspace{0.75em}0.36^{}$,$\hspace{0.75em}0.84^{}$,$-5.58^{***}$,$\hspace{0.75em}1.06^{}$,$-1.72^{*}$,$-1.62^{***}$,$\hspace{0.75em}2.2^{*}$,$\hspace{0.75em}3.02^{***}$,$\hspace{0.75em}0.98^{}$
MRPC STSB,bert,$-1.64^{}$,$\hspace{0.75em}0.36^{}$,$-0.1^{}$,$\hspace{0.75em}0.12^{}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}0.8^{}$,$-0.16^{}$,$\hspace{0.75em}2.9^{*}$,$\hspace{0.75em}4.4^{***}$
COLA MRPC,bert,$\hspace{0.75em}1.08^{}$,$\hspace{0.75em}0.82^{}$,$-2.7^{}$,$\hspace{0.75em}0.34^{}$,$-0.14^{}$,$-1.46^{}$,$\hspace{0.75em}2.36^{}$,$\hspace{0.75em}0.02^{}$,$\hspace{0.75em}1.6^{}$
SST2 MRPC,bert,$\hspace{0.75em}1.02^{}$,$\hspace{0.75em}0.52^{}$,$-2.84^{}$,$\hspace{0.75em}0.5^{}$,$-0.64^{}$,$-1.02^{}$,$\hspace{0.75em}0.58^{}$,$\hspace{0.75em}0.92^{}$,$\hspace{0.75em}1.14^{}$
MRPC QNLI,bert,$\hspace{0.75em}0.08^{}$,$-0.26^{}$,$-1.16^{}$,$\hspace{0.75em}1.56^{}$,$\hspace{0.75em}0.04^{}$,$-0.58^{}$,$\hspace{0.75em}1.26^{}$,$\hspace{0.75em}1.52^{}$,$\hspace{0.75em}0.62^{}$
MRPC RTE,bert,$\hspace{0.75em}2.74^{}$,$\hspace{0.75em}0.36^{}$,$-5.24^{***}$,$\hspace{0.75em}0.52^{}$,$-0.3^{}$,$-4.0^{*}$,$-0.14^{}$,$\hspace{0.75em}0.52^{}$,$\hspace{0.75em}0.66^{}$
COLA MRPC QNLI,bert,$-0.54^{}$,$\hspace{0.75em}1.46^{}$,$-2.76^{}$,$-0.42^{}$,$-0.8^{*}$,$-0.24^{}$,$\hspace{0.75em}1.46^{}$,$\hspace{0.75em}0.18^{}$,$\hspace{0.75em}0.32^{}$
COLA MRPC RTE,bert,$\hspace{0.75em}0.66^{}$,$\hspace{0.75em}1.54^{*}$,$-1.02^{}$,$\hspace{0.75em}0.1^{}$,$-0.36^{}$,$-0.64^{}$,$\hspace{0.75em}0.3^{}$,$-0.18^{}$,$\hspace{0.75em}0.72^{}$
SST2 MRPC QNLI,bert,$\hspace{0.75em}1.78^{}$,$\hspace{0.75em}0.36^{}$,$-1.2^{}$,$\hspace{0.75em}0.9^{}$,$-0.42^{}$,$\hspace{0.75em}0.68^{}$,$\hspace{0.75em}2.6^{*}$,$\hspace{0.75em}1.84^{}$,$\hspace{0.75em}1.32^{}$
SST2 MRPC RTE,bert,$\hspace{0.75em}0.98^{}$,$\hspace{0.75em}1.48^{}$,$-0.78^{}$,$\hspace{0.75em}0.82^{}$,$-0.64^{}$,$\hspace{0.75em}0.32^{}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}0.52^{}$,$\hspace{0.75em}1.6^{}$


## RoBERTa

In [16]:
mrpc_df_roberta = mrpc_df[mrpc_df["LM"]=="roberta"]
mrpc_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
MRPC,roberta,$\hspace{0.75em}4.02^{*}$,$\hspace{0.75em}1.4^{*}$,$\hspace{0.75em}4.28^{*}$,$-4.02^{***}$,$-1.32^{*}$,$-1.86^{}$,$-1.98^{}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}1.76^{*}$
MRPC STSB,roberta,$\hspace{0.75em}0.68^{}$,$-1.74^{*}$,$\hspace{0.75em}3.2^{}$,$-0.18^{}$,$\hspace{0.75em}2.26^{}$,$\hspace{0.75em}1.12^{}$,$-1.48^{}$,$-1.42^{}$,$\hspace{0.75em}0.98^{}$
COLA MRPC,roberta,$-3.64^{*}$,$-0.18^{}$,$-5.92^{***}$,$-1.86^{***}$,$-1.46^{*}$,$-2.14^{***}$,$\hspace{0.75em}0.58^{}$,$-1.52^{*}$,$\hspace{0.75em}0.66^{}$
SST2 MRPC,roberta,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}2.36^{}$,$\hspace{0.75em}1.56^{}$,$\hspace{0.75em}1.62^{}$,$\hspace{0.75em}0.66^{}$,$\hspace{0.75em}2.54^{}$,$\hspace{0.75em}1.76^{}$,$\hspace{0.75em}1.38^{}$,$\hspace{0.75em}1.26^{}$
MRPC QNLI,roberta,$\hspace{0.75em}1.98^{}$,$\hspace{0.75em}0.26^{}$,$-0.16^{}$,$-3.38^{*}$,$-0.9^{}$,$-1.54^{}$,$-0.54^{}$,$\hspace{0.75em}1.4^{}$,$\hspace{0.75em}2.82^{*}$
MRPC RTE,roberta,$-2.56^{}$,$-0.18^{}$,$\hspace{0.75em}1.18^{}$,$-2.14^{*}$,$-2.8^{}$,$-1.08^{}$,$-1.94^{}$,$\hspace{0.75em}0.4^{}$,$\hspace{0.75em}3.48^{*}$
COLA MRPC QNLI,roberta,$\hspace{0.75em}0.08^{}$,$\hspace{0.75em}0.06^{}$,$-0.3^{}$,$-0.44^{}$,$-0.02^{}$,$\hspace{0.75em}1.34^{}$,$\hspace{0.75em}1.28^{}$,$-0.26^{}$,$\hspace{0.75em}0.34^{}$
COLA MRPC RTE,roberta,$-0.18^{}$,$\hspace{0.75em}0.12^{}$,$-1.76^{}$,$-1.28^{}$,$\hspace{0.75em}0.16^{}$,$\hspace{0.75em}0.96^{}$,$\hspace{0.75em}0.32^{}$,$-0.48^{}$,$-0.18^{}$
SST2 MRPC QNLI,roberta,$\hspace{0.75em}0.14^{}$,$-0.18^{}$,$-2.58^{}$,$-1.34^{}$,$-1.96^{}$,$\hspace{0.75em}0.86^{}$,$\hspace{0.75em}1.3^{}$,$-0.08^{}$,$\hspace{0.75em}1.48^{}$
SST2 MRPC RTE,roberta,$-1.36^{}$,$\hspace{0.75em}0.6^{}$,$-8.22^{***}$,$-3.76^{*}$,$-4.7^{***}$,$-6.32^{*}$,$-3.82^{*}$,$\hspace{0.75em}0.76^{}$,$-2.62^{}$


# STSB

## BERT

In [17]:
stsb_df = ind_effect_task_dfs_dict["stsb"]
stsb_df_bert = stsb_df[stsb_df["LM"]=="bert"]
stsb_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
STSB,bert,$\hspace{0.75em}0.2^{}$,$\hspace{0.75em}0.9^{}$,$-4.16^{***}$,$-0.02^{}$,$-1.9^{***}$,$\hspace{0.75em}0.4^{}$,$\hspace{0.75em}2.96^{***}$,$\hspace{0.75em}0.34^{}$,$-2.42^{***}$
MRPC STSB,bert,$-1.8^{}$,$\hspace{0.75em}0.42^{}$,$\hspace{0.75em}1.32^{}$,$-0.96^{}$,$\hspace{0.75em}0.32^{}$,$\hspace{0.75em}2.82^{*}$,$\hspace{0.75em}0.6^{}$,$\hspace{0.75em}0.22^{}$,$\hspace{0.75em}1.0^{}$
COLA STSB,bert,$-1.32^{}$,$-0.9^{}$,$-2.02^{}$,$\hspace{0.75em}0.14^{}$,$\hspace{0.75em}0.18^{}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}1.36^{}$,$\hspace{0.75em}0.12^{}$,$\hspace{0.75em}0.82^{}$
SST2 STSB,bert,$-0.5^{}$,$-0.0^{}$,$-2.66^{}$,$-0.84^{}$,$-0.78^{}$,$-1.16^{}$,$-0.7^{}$,$-0.18^{}$,$\hspace{0.75em}0.38^{}$
STSB QNLI,bert,$-0.28^{}$,$\hspace{0.75em}0.32^{}$,$-2.88^{}$,$\hspace{0.75em}0.44^{}$,$-0.46^{}$,$\hspace{0.75em}0.54^{}$,$-0.22^{}$,$\hspace{0.75em}1.3^{}$,$-0.72^{}$
STSB RTE,bert,$\hspace{0.75em}1.26^{}$,$-0.98^{}$,$-3.86^{***}$,$-0.84^{}$,$-0.2^{}$,$-1.2^{}$,$-0.72^{}$,$\hspace{0.75em}0.3^{}$,$-0.22^{}$
COLA STSB QNLI,bert,$-1.84^{}$,$\hspace{0.75em}1.06^{}$,$-0.5^{}$,$-0.2^{}$,$\hspace{0.75em}0.02^{}$,$\hspace{0.75em}0.24^{}$,$0.0^{}$,$\hspace{0.75em}0.22^{}$,$-0.92^{}$
COLA STSB RTE,bert,$-0.42^{}$,$\hspace{0.75em}0.58^{}$,$\hspace{0.75em}0.36^{}$,$\hspace{0.75em}0.46^{}$,$-0.14^{}$,$\hspace{0.75em}0.12^{}$,$-1.26^{}$,$-0.78^{}$,$-0.1^{}$
SST2 STSB QNLI,bert,$\hspace{0.75em}0.88^{}$,$\hspace{0.75em}1.34^{}$,$-0.66^{}$,$\hspace{0.75em}0.06^{}$,$\hspace{0.75em}0.1^{}$,$\hspace{0.75em}1.56^{}$,$\hspace{0.75em}1.22^{}$,$\hspace{0.75em}2.72^{*}$,$\hspace{0.75em}1.36^{}$
SST2 STSB RTE,bert,$-1.46^{}$,$-0.06^{}$,$-1.56^{}$,$-0.6^{}$,$-0.18^{}$,$-0.24^{}$,$-1.2^{}$,$-0.68^{}$,$-0.04^{}$


## RoBERTa

In [18]:
stsb_df_roberta = stsb_df[stsb_df["LM"]=="roberta"]
stsb_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
STSB,roberta,$\hspace{0.75em}1.92^{*}$,$\hspace{0.75em}1.92^{*}$,$-1.4^{}$,$-6.78^{***}$,$-4.52^{***}$,$-4.4^{***}$,$-3.42^{***}$,$-0.42^{}$,$\hspace{0.75em}0.36^{}$
MRPC STSB,roberta,$-1.42^{}$,$-1.22^{}$,$-2.48^{}$,$-2.94^{}$,$-0.94^{}$,$-1.42^{}$,$-2.92^{}$,$-2.34^{}$,$-0.42^{}$
COLA STSB,roberta,$-4.1^{*}$,$-0.32^{}$,$-6.22^{*}$,$-0.66^{}$,$-1.34^{*}$,$-3.86^{***}$,$-1.74^{}$,$-0.46^{}$,$-0.14^{}$
SST2 STSB,roberta,$-0.94^{}$,$\hspace{0.75em}1.98^{}$,$-1.04^{}$,$-0.14^{}$,$-0.12^{}$,$-0.2^{}$,$\hspace{0.75em}0.18^{}$,$\hspace{0.75em}1.02^{}$,$\hspace{0.75em}1.8^{}$
STSB QNLI,roberta,$\hspace{0.75em}2.04^{}$,$-0.5^{}$,$-4.5^{}$,$-5.6^{***}$,$-1.74^{}$,$-3.86^{*}$,$-2.64^{*}$,$-0.28^{}$,$\hspace{0.75em}1.52^{}$
STSB RTE,roberta,$\hspace{0.75em}3.04^{*}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}4.88^{}$,$-3.5^{*}$,$-1.4^{}$,$-2.88^{}$,$-1.58^{}$,$-0.16^{}$,$\hspace{0.75em}2.36^{}$
COLA STSB QNLI,roberta,$\hspace{0.75em}1.68^{}$,$\hspace{0.75em}0.38^{}$,$-4.0^{***}$,$-0.44^{}$,$-0.42^{}$,$-1.7^{}$,$-0.4^{}$,$-0.72^{}$,$-1.4^{}$
COLA STSB RTE,roberta,$-0.26^{}$,$-1.34^{*}$,$-7.14^{*}$,$-0.42^{}$,$-1.54^{}$,$-5.12^{*}$,$-1.24^{}$,$-0.88^{}$,$-1.5^{}$
SST2 STSB QNLI,roberta,$-0.54^{}$,$\hspace{0.75em}0.46^{}$,$-2.48^{}$,$-4.88^{}$,$-2.08^{}$,$-2.02^{}$,$-0.84^{}$,$-0.52^{}$,$\hspace{0.75em}0.52^{}$
SST2 STSB RTE,roberta,$-1.08^{}$,$-0.48^{}$,$-7.28^{*}$,$-4.66^{***}$,$-4.26^{*}$,$-5.66^{*}$,$-1.88^{}$,$\hspace{0.75em}0.48^{}$,$-1.48^{}$


# QNLI

## BERT

In [19]:
qnli_df = ind_effect_task_dfs_dict["qnli"]
qnli_df_bert = qnli_df[qnli_df["LM"]=="bert"]
qnli_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
QNLI,bert,$-1.1^{}$,$\hspace{0.75em}1.04^{}$,$-0.54^{}$,$-1.02^{}$,$-1.02^{***}$,$\hspace{0.75em}1.46^{}$,$\hspace{0.75em}1.62^{*}$,$\hspace{0.75em}0.48^{}$,$\hspace{0.75em}1.8^{***}$
QNLI RTE,bert,$\hspace{0.75em}2.36^{}$,$-0.28^{}$,$-2.66^{*}$,$-0.9^{}$,$-0.66^{}$,$-1.92^{}$,$-1.14^{}$,$\hspace{0.75em}1.18^{}$,$\hspace{0.75em}0.6^{}$
COLA QNLI,bert,$\hspace{0.75em}1.46^{}$,$-1.02^{}$,$\hspace{0.75em}1.14^{}$,$-0.14^{}$,$-0.12^{}$,$\hspace{0.75em}0.52^{}$,$\hspace{0.75em}1.3^{}$,$-0.62^{}$,$\hspace{0.75em}2.08^{}$
SST2 QNLI,bert,$-0.74^{}$,$\hspace{0.75em}0.46^{}$,$-0.48^{}$,$-0.56^{}$,$-0.34^{}$,$-0.84^{}$,$-1.18^{}$,$-2.56^{***}$,$\hspace{0.75em}0.52^{}$
MRPC QNLI,bert,$-1.38^{}$,$-0.06^{}$,$\hspace{0.75em}3.88^{*}$,$-0.52^{}$,$\hspace{0.75em}0.74^{}$,$\hspace{0.75em}2.5^{*}$,$\hspace{0.75em}0.68^{}$,$-1.02^{}$,$\hspace{0.75em}1.44^{}$
STSB QNLI,bert,$-1.58^{}$,$\hspace{0.75em}0.46^{}$,$\hspace{0.75em}0.74^{}$,$-0.56^{}$,$\hspace{0.75em}0.42^{}$,$\hspace{0.75em}1.6^{}$,$-1.56^{}$,$\hspace{0.75em}1.44^{}$,$\hspace{0.75em}3.5^{***}$
COLA MRPC QNLI,bert,$-0.16^{}$,$-0.38^{}$,$\hspace{0.75em}1.08^{}$,$-0.9^{}$,$-0.78^{*}$,$\hspace{0.75em}1.74^{}$,$\hspace{0.75em}0.4^{}$,$-0.46^{}$,$\hspace{0.75em}0.8^{}$
COLA STSB QNLI,bert,$\hspace{0.75em}0.94^{}$,$\hspace{0.75em}0.94^{}$,$\hspace{0.75em}2.66^{}$,$-0.48^{}$,$-0.28^{}$,$\hspace{0.75em}0.26^{}$,$-0.06^{}$,$-0.52^{}$,$\hspace{0.75em}0.34^{}$
SST2 MRPC QNLI,bert,$\hspace{0.75em}0.02^{}$,$\hspace{0.75em}0.3^{}$,$\hspace{0.75em}1.16^{}$,$-0.16^{}$,$-0.12^{}$,$\hspace{0.75em}0.86^{}$,$\hspace{0.75em}0.84^{}$,$-1.64^{}$,$\hspace{0.75em}0.7^{}$
SST2 STSB QNLI,bert,$\hspace{0.75em}0.64^{}$,$\hspace{0.75em}1.8^{*}$,$\hspace{0.75em}1.52^{}$,$\hspace{0.75em}0.34^{}$,$\hspace{0.75em}0.54^{}$,$\hspace{0.75em}1.88^{}$,$\hspace{0.75em}0.74^{}$,$\hspace{0.75em}0.34^{}$,$\hspace{0.75em}1.5^{}$


## RoBERTa

In [20]:
qnli_df_roberta = qnli_df[qnli_df["LM"]=="roberta"]
qnli_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
QNLI,roberta,$\hspace{0.75em}0.26^{}$,$\hspace{0.75em}1.0^{*}$,$\hspace{0.75em}2.88^{}$,$-0.9^{}$,$-1.34^{}$,$-0.78^{}$,$-3.28^{***}$,$\hspace{0.75em}0.62^{}$,$-0.96^{}$
QNLI RTE,roberta,$\hspace{0.75em}0.16^{}$,$\hspace{0.75em}0.26^{}$,$\hspace{0.75em}2.2^{}$,$\hspace{0.75em}0.06^{}$,$-1.28^{}$,$-1.54^{}$,$-1.98^{}$,$\hspace{0.75em}1.02^{}$,$\hspace{0.75em}0.9^{}$
COLA QNLI,roberta,$-5.6^{*}$,$-0.74^{}$,$-5.36^{***}$,$-0.82^{}$,$-1.72^{}$,$-3.1^{}$,$-2.46^{}$,$-0.18^{}$,$-0.64^{}$
SST2 QNLI,roberta,$\hspace{0.75em}0.44^{}$,$\hspace{0.75em}2.02^{}$,$\hspace{0.75em}3.06^{}$,$\hspace{0.75em}1.22^{}$,$\hspace{0.75em}1.3^{}$,$\hspace{0.75em}2.3^{}$,$\hspace{0.75em}0.56^{}$,$\hspace{0.75em}1.22^{}$,$\hspace{0.75em}1.86^{}$
MRPC QNLI,roberta,$-1.78^{}$,$-0.14^{}$,$-1.56^{}$,$-0.26^{}$,$-0.92^{}$,$-0.46^{}$,$-1.84^{}$,$\hspace{0.75em}1.52^{}$,$\hspace{0.75em}0.1^{}$
STSB QNLI,roberta,$\hspace{0.75em}0.38^{}$,$-1.42^{}$,$-0.22^{}$,$\hspace{0.75em}0.28^{}$,$\hspace{0.75em}1.44^{}$,$-0.24^{}$,$-2.5^{*}$,$\hspace{0.75em}0.76^{}$,$\hspace{0.75em}0.2^{}$
COLA MRPC QNLI,roberta,$-1.88^{}$,$-0.5^{}$,$\hspace{0.75em}0.26^{}$,$\hspace{0.75em}0.6^{}$,$-0.28^{}$,$\hspace{0.75em}0.38^{}$,$-1.76^{}$,$\hspace{0.75em}1.08^{}$,$-0.96^{}$
COLA STSB QNLI,roberta,$\hspace{0.75em}0.18^{}$,$-0.04^{}$,$-3.14^{}$,$-0.6^{}$,$-0.8^{}$,$-0.94^{}$,$-1.12^{}$,$-0.44^{}$,$-1.9^{}$
SST2 MRPC QNLI,roberta,$\hspace{0.75em}0.3^{}$,$-0.52^{}$,$-1.08^{}$,$-1.74^{}$,$-1.32^{}$,$\hspace{0.75em}0.62^{}$,$\hspace{0.75em}0.1^{}$,$-0.24^{}$,$\hspace{0.75em}2.08^{}$
SST2 STSB QNLI,roberta,$\hspace{0.75em}0.84^{}$,$\hspace{0.75em}0.5^{}$,$\hspace{0.75em}1.62^{}$,$-3.52^{}$,$-0.66^{}$,$\hspace{0.75em}0.48^{}$,$-0.46^{}$,$-0.32^{}$,$\hspace{0.75em}0.58^{}$


# RTE

## BERT

In [21]:
rte_df = ind_effect_task_dfs_dict["rte"]
rte_df_bert = rte_df[rte_df["LM"]=="bert"]
rte_df_bert

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
RTE,bert,$-1.2^{}$,$\hspace{0.75em}1.86^{***}$,$-0.44^{}$,$\hspace{0.75em}0.12^{}$,$-1.02^{*}$,$\hspace{0.75em}1.8^{}$,$\hspace{0.75em}1.48^{***}$,$\hspace{0.75em}1.74^{}$,$\hspace{0.75em}0.94^{}$
QNLI RTE,bert,$\hspace{0.75em}2.26^{}$,$\hspace{0.75em}0.54^{}$,$-2.56^{*}$,$\hspace{0.75em}0.24^{}$,$-0.66^{*}$,$-1.58^{}$,$-1.28^{}$,$\hspace{0.75em}2.44^{}$,$-0.26^{}$
COLA RTE,bert,$\hspace{0.75em}1.62^{}$,$-0.76^{}$,$-0.8^{}$,$-0.26^{}$,$-0.14^{}$,$-0.2^{}$,$\hspace{0.75em}1.14^{}$,$\hspace{0.75em}0.34^{}$,$\hspace{0.75em}1.6^{}$
SST2 RTE,bert,$\hspace{0.75em}0.62^{}$,$-0.4^{}$,$-2.46^{}$,$-0.5^{}$,$-0.38^{}$,$-1.16^{}$,$-0.52^{}$,$-0.86^{}$,$-0.36^{}$
MRPC RTE,bert,$\hspace{0.75em}1.18^{}$,$\hspace{0.75em}1.38^{*}$,$-0.1^{}$,$-0.42^{}$,$\hspace{0.75em}0.4^{}$,$-0.58^{}$,$-0.86^{}$,$-0.76^{}$,$\hspace{0.75em}0.62^{}$
STSB RTE,bert,$-0.14^{}$,$-0.02^{}$,$-0.14^{}$,$-0.7^{}$,$\hspace{0.75em}0.68^{}$,$\hspace{0.75em}0.2^{}$,$-2.2^{*}$,$\hspace{0.75em}1.7^{}$,$\hspace{0.75em}3.14^{*}$
COLA MRPC RTE,bert,$\hspace{0.75em}1.2^{}$,$-0.04^{}$,$\hspace{0.75em}0.88^{}$,$-0.5^{}$,$-0.36^{}$,$\hspace{0.75em}0.62^{}$,$-0.92^{}$,$\hspace{0.75em}0.14^{}$,$\hspace{0.75em}0.72^{}$
COLA STSB RTE,bert,$\hspace{0.75em}2.52^{*}$,$\hspace{0.75em}0.72^{}$,$\hspace{0.75em}1.58^{}$,$\hspace{0.75em}0.06^{}$,$-0.46^{}$,$-0.58^{}$,$-1.48^{}$,$-0.56^{}$,$\hspace{0.75em}0.68^{}$
SST2 MRPC RTE,bert,$\hspace{0.75em}0.58^{}$,$\hspace{0.75em}0.56^{}$,$-0.4^{}$,$-0.18^{}$,$-0.38^{}$,$\hspace{0.75em}0.18^{}$,$-0.82^{}$,$-1.26^{}$,$\hspace{0.75em}0.1^{}$
SST2 STSB RTE,bert,$-0.34^{}$,$-0.46^{}$,$-1.36^{}$,$-0.26^{}$,$\hspace{0.75em}0.22^{}$,$-0.24^{}$,$-1.02^{}$,$-1.36^{}$,$-0.78^{}$


## RoBERTa

In [22]:
rte_df_roberta = rte_df[rte_df["LM"]=="roberta"]
rte_df_roberta

,LM,Length,Depth,TopConst,BigramShift,Tense,SubjNumber,ObjNumber,OddManOut,CoordInv
RTE,roberta,$\hspace{0.75em}2.04^{*}$,$-0.36^{}$,$-1.22^{}$,$-2.36^{***}$,$-1.6^{}$,$-1.14^{}$,$-3.88^{***}$,$-0.26^{}$,$-1.88^{*}$
QNLI RTE,roberta,$\hspace{0.75em}1.94^{}$,$-1.1^{}$,$-1.9^{}$,$-1.4^{}$,$-1.54^{}$,$-1.9^{}$,$-2.58^{*}$,$\hspace{0.75em}0.14^{}$,$-0.02^{}$
COLA RTE,roberta,$-1.96^{}$,$-0.56^{}$,$-3.8^{}$,$-0.36^{}$,$-1.52^{}$,$-2.08^{*}$,$-1.82^{*}$,$-0.72^{}$,$-0.3^{}$
SST2 RTE,roberta,$\hspace{0.75em}1.84^{}$,$\hspace{0.75em}1.12^{}$,$\hspace{0.75em}6.92^{***}$,$\hspace{0.75em}3.42^{*}$,$\hspace{0.75em}3.08^{*}$,$\hspace{0.75em}4.42^{*}$,$\hspace{0.75em}1.6^{}$,$\hspace{0.75em}0.24^{}$,$\hspace{0.75em}4.08^{***}$
MRPC RTE,roberta,$-4.54^{*}$,$-1.94^{***}$,$-4.32^{}$,$-0.48^{}$,$-3.08^{*}$,$-0.36^{}$,$-3.84^{*}$,$-0.36^{}$,$-0.16^{}$
STSB RTE,roberta,$\hspace{0.75em}3.16^{*}$,$-1.78^{*}$,$\hspace{0.75em}5.06^{}$,$\hspace{0.75em}0.92^{}$,$\hspace{0.75em}1.52^{}$,$\hspace{0.75em}0.38^{}$,$-2.04^{}$,$0.0^{}$,$\hspace{0.75em}0.12^{}$
COLA MRPC RTE,roberta,$\hspace{0.75em}1.5^{}$,$-0.26^{}$,$\hspace{0.75em}0.36^{}$,$\hspace{0.75em}0.22^{}$,$\hspace{0.75em}0.1^{}$,$\hspace{0.75em}1.02^{}$,$-2.08^{}$,$\hspace{0.75em}0.32^{}$,$-1.14^{}$
COLA STSB RTE,roberta,$\hspace{0.75em}1.88^{}$,$-1.58^{*}$,$-4.72^{}$,$-0.12^{}$,$-1.72^{}$,$-3.34^{}$,$-1.32^{}$,$-1.14^{}$,$-1.66^{}$
SST2 MRPC RTE,roberta,$\hspace{0.75em}0.2^{}$,$-0.64^{}$,$-2.86^{}$,$-1.96^{}$,$-2.28^{}$,$-4.44^{}$,$-3.98^{*}$,$-0.38^{}$,$\hspace{0.75em}0.2^{}$
SST2 STSB RTE,roberta,$\hspace{0.75em}1.7^{}$,$-1.34^{*}$,$\hspace{0.75em}0.68^{}$,$-1.1^{}$,$-1.06^{}$,$-1.04^{}$,$-0.46^{}$,$-0.3^{}$,$\hspace{0.75em}0.8^{}$
